In [1]:
import requests
import json

url = "https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001"
params = {
    "Authorization": "CWB-11505125-6C17-4A38-8873-3D1B3214BCD8", #API授權碼
    "locationName": "桃園市",  #城市
}

response = requests.get(url, params=params)
print(response.status_code)

#將資料撈出來看看
if response.status_code == 200:
    # print(response.text)
    data = json.loads(response.text)

    location = data["records"]["location"][0]["locationName"]

    weather_elements = data["records"]["location"][0]["weatherElement"]
    start_time = weather_elements[0]["time"][1]["startTime"]
    end_time = weather_elements[0]["time"][1]["endTime"]
    weather_state = weather_elements[0]["time"][1]["parameter"]["parameterName"]
    rain_prob = weather_elements[1]["time"][1]["parameter"]["parameterName"]
    min_tem = weather_elements[2]["time"][1]["parameter"]["parameterName"]
    comfort = weather_elements[3]["time"][1]["parameter"]["parameterName"]
    max_tem = weather_elements[4]["time"][1]["parameter"]["parameterName"]

    print(location)
    print(start_time)
    print(end_time)
    print(weather_state)
    print(rain_prob)
    print(min_tem)
    print(comfort)
    print(max_tem)

else:
    print("Can't get data!")

200
桃園市
2022-06-17 06:00:00
2022-06-17 18:00:00
多雲時陰
20
25
舒適至悶熱
30


In [2]:
#傳送訊息
def line_notify(data):

    token = "HfdnpRgbEQ6K0LIJMKQ0AfYCazLOmHHWLfPS5I6CRZ4"  #LINE Notify權杖
    message = ""

    if len(data) == 0:
        message += "\n[Error] 無法取得天氣資訊"
    else:
        message += f"\n今天{data[0]}的天氣: {data[3]}\n"
        message += f"溫度: {data[5]}°C - {data[7]}°C\n"
        message += f"降雨機率: {data[4]}%\n"
        message += f"舒適度: {data[6]}\n"
        message += f"時間: {data[1]} ~ {data[2]}\n"


        image = open('pre.jfif', 'rb')    # 以二進位方式開啟圖片
        imageFile = {'imageFile' : image}   # 設定圖片資訊
        
        if int(data[4]) > 70:
            message += "系女排雨神同行，今天又要停練了.."
            image = open('rain.jpg', 'rb')    # 以二進位方式開啟圖片
            imageFile = {'imageFile' : image}   # 設定圖片資訊
        elif int(data[7]) > 30:
            message += "太陽公公出來了，不擦防曬會變黑人~"
            image = open('sun.jfif', 'rb')    # 以二進位方式開啟圖片
            imageFile = {'imageFile' : image}   # 設定圖片資訊
        elif int(data[5]) < 15:
            message += "今天很冷，穿短褲膝蓋會壞掉！"
            image = open('cold.jpg', 'rb')    # 以二進位方式開啟圖片
            imageFile = {'imageFile' : image}   # 設定圖片資訊

    # line notify所需資料
    line_url = "https://notify-api.line.me/api/notify"
    line_header = {
        "Authorization": 'Bearer ' + token
    }
    line_data = {
        "message": message
    }

    line_data = requests.post(url=line_url, headers=line_header, data=line_data, files=imageFile)

In [3]:
#呼叫line_notify
line_notify(tuple([location, start_time, end_time, weather_state, rain_prob, min_tem, comfort, max_tem]))